In [18]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import pandas as pd

In [3]:
cities = 'London ,Manchester,Birmingham,Wolverhampton ,Leeds, Bradford ,Glasgow ,Southampton,Portsmouth ,Liverpool ,Newcastle ,Nottingham ,Sheffield,Bristol,Belfast,Leicester'
city_lst = []
for i in cities.split(','):
    city_lst.append(i.strip())
    
print(city_lst)

['London', 'Manchester', 'Birmingham', 'Wolverhampton', 'Leeds', 'Bradford', 'Glasgow', 'Southampton', 'Portsmouth', 'Liverpool', 'Newcastle', 'Nottingham', 'Sheffield', 'Bristol', 'Belfast', 'Leicester']


In [4]:
url = 'https://www.indeed.co.uk/jobs?q=junior+data+scientist&l=london&start=0'
# url = 'https://www.indeed.co.uk/jobs?q=junior+data+scientist&l=london&start=10' # worked fine
r = requests.get(url)
r.status_code

200

In [5]:
# Create soup and have a glimpse
soup = BeautifulSoup(r.text, 'html.parser')
# print(soup.prettify())

1.注意这里的 page 变量的使用。
2.整个过程其实可以写成一个function。

In [12]:
titles = []
for page in range(0, 100, 10):
    url = 'https://www.indeed.co.uk/jobs?q=junior+data+scientist&l=london&start={}'.format(page)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    for title in soup.find_all('a', attrs={'data-tn-element':'jobTitle'}):
        try:
            titles.append(title.text.strip())
        except:
            titles.append('No content')

In [16]:
print(len(titles))
print(titles)

141
['Junior Machine Learning Researcher / Data Scientist', 'Junior Data Scientist', 'Junior Data Analyst', 'Junior Data Scientist', 'Junior Machine Learning Researcher / Data Scientist', 'Data Scientist - ML Labs', 'Junior Marketing Executive', 'Sr. Data Scientist - ML Labs', 'Junior Data Scientist - London', 'Data Scientist', 'Data Scientist', 'Principal Data Scientist', 'Junior Product Manager - London', 'Senior Data Scientist', 'Senior Data Scientist', 'Data Scientist (Mid - Lead level)', 'AI-Quant - Junior', 'Data & Analytics Consultant (m/f/d)', 'Characterisation Scientist', 'Data Scientist', 'Senior Data Scientist (FTC - Maternity Cover)', 'Research Scientist – Evidence Synthesis', 'Software Engineer - Junior', 'Data & Analytics Consultant', 'Lead Data Scientist - NLP', 'Senior Data Scientist', 'Lead Data Scientist', 'Scientist, Cell & Gene Therapy Technologies', 'Data Scientist', 'Data Engineer', 'G7 Economists *Closing Date Extended*', 'Senior Data Scientist - Digital', 'Senio

In [14]:
salaries = []
for page in range(0, 100, 10):
    url = 'https://www.indeed.co.uk/jobs?q=junior+data+scientist&l=london&start={}'.format(page)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    for title in soup.find_all("span","salary no-wrap"):
        try:
            salaries.append(title.text.strip())
        except:
            salaries.append('No content')

In [17]:
print(len(salaries))
print(salaries)

33
['£25,000 - £35,000 a year', '£50,000 - £110,000 a year', '£135,000 a year', '£65,000 - £70,000 a year', '£75,000 a year', '£47,078 - £59,889 a year', '£43,650 - £69,700 a year', '£80,000 - £90,000 a year', '£80,000 - £95,000 a year', '£135,000 a year', '£40,000 - £50,000 a year', '£90,000 a year', '£40,000 - £50,000 a year', '£60,000 - £70,000 a year', '£70,000 a year', '£70,000 a year', '£80,000 - £85,000 a year', '£500 - £650 a week', '£75,000 - £90,000 a year', '£40,000 - £55,000 a year', '£70,000 - £80,000 a year', '£33,797 - £40,322 a year', '£70,000 - £75,000 a year', '£80,000 - £95,000 a year', '£75,000 - £90,000 a year', '£33,797 - £40,322 a year', '£70,000 - £75,000 a year', '£80,000 - £95,000 a year', '£75,000 - £90,000 a year', '£33,797 - £40,322 a year', '£70,000 - £75,000 a year', '£80,000 - £95,000 a year', '£75,000 - £90,000 a year']


需求：把上面的titles和salaries装入一个pandas表格，让title和salary一一对应。

In [20]:
# results = pd.DataFrame({'title':titles, 'salary':salaries})
# results

In [ ]:
# continue tomorrow

In [ ]:
#sdfasdf

### companies, salaries, locations, job descriptions

In [8]:
# bs = soup.find_all('span', attrs={'class':'company'})
# for title in bs:
#     print(title.text.strip())

In [9]:
# money = []
# bs = soup.find_all("span","salary no-wrap")
# for title in bs:
#     if len(title.text) >3  :
#         money.append(title.text.strip())
#     else:
#         money.append('0')

In [10]:
# bs = soup.find_all('span', attrs={'class':'location'})
# for title in bs:
#     print(title.text.strip())

In [11]:
# bs = soup.find_all("div",{"class":"summary"})
# for title in bs:
#     print(title.text.strip())